# Tema 6 - Pamfile Alex

## Modele de regresie

Selectati 4 seturi de date de la una din adresele de mai jos:
1. [https://www.dcc.fc.up.pt/~ltorgo/Regression/DataSets.html](https://www.dcc.fc.up.pt/~ltorgo/Regression/DataSets.html)
1. https://data.world/datasets/regression
1. https://homepages.inf.ed.ac.uk/rbf/IAPR/researchers/MLPAGES/mldat.htm
1. https://data.world/nrippner/ols-regression-challenge
1. https://www.interviewquery.com/p/regression-datasets-and-projects

Excludeti seturile de date: Machine CPU, Boston Housing, Wisconsin Breast Cancer, Communities and Crime.

Acolo unde este cazul, efectuati missing value imputation, de exemplu cu metodele indicate in Laboratorul 6. Metoda de completare va fi inclusa intr-un pipeline (a se vedea cursul 8). 

Pentru fiecare set de date aplicati minim 5 modele de regresie din scikit learn. Pentru fiecare raportati: mean absolute error, mean squared error, median absolute error - referinta [sklearn.metrics](http://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics) - folosind 5 fold cross validation. Valorile hiperparametrilor trebuie cautate cu grid search (cv=3)  si random search (n_iter dat de voi). Metrica folosita pentru optimizarea hiperparametrilor va fi mean squared error. Raportati mediile rezultatelor atat pentru fold-urile de antrenare, cat si pentru cele de testare; indicatie: puteti folosi metoda `cross_validate` cu parametrul `return_train_score=True`, iar ca model un obiect de tip `GridSearchCV` sau `RandomizedSearchCV`.

Rezultatele vor fi trecute intr-un dataframe. Intr-o stare intermediara, valorile vor fi calculate cu semnul minus: din motive de implementare, biblioteca sklearn transforma scorurile in numere negative; a se vedea imaginea de mai jos:

![intermediate report](./images/cpu_intermediate_blurred.png)


Valorile vor fi aduse la interval pozitiv, apoi vor fi marcate cele maxime si minime; orientativ, se poate folosi imaginea de mai jos, reprezentand dataframe afisat in notebook; puteti folosi alte variante de styling pe dataframe precum la https://pandas.pydata.org/pandas-docs/stable/user_guide/style.html#.  

Se va crea un raport final in format HTML sau PDF - fisier(e) separat(e). Raportul trebuie sa contina minimal: numele setului de date si obiectul dataframe; preferabil sa se pastreze marcajul de culori realizat in notebook.

![report](./images/cpu_results_blurred.png)

Notare:
1. Se acorda 20 de puncte din oficiu.
1. Optimizare si cuantificare de performanta a modelelor: 3 puncte pentru fiecare combinatie set de date + model = 60 de puncte
1. Documentare modele: numar modele * 2 puncte = 10 puncte. Documentati in jupyter notebook fiecare din modelele folosite, in limba romana. Puteti face o sectiune separata cu documentarea algoritmilor. Fiecare model trebuie sa aiba o descriere de minim 20 de randuri, minim o imagine asociata si minim 2 referinte bibliografice.
1. 10 puncte: export in format HTML sau PDF.



## Precizari

* Se va face o sectiune separata in cadrul notebook-ului pentru fiecare sursa de date. 
* Asigurati-va ca includeti fisierele de date folosite in arhiva predata
* Pot fi folosite aceleasi modele de regresie pentru toate seturile de date.
* Depunerea se face in Tema 6 pe elearning, pana cel tarziu 13 decembrie ora 23.
* Specificatiile legate de depunere, numele fisierelor etc. sunt aceleasi ca la tema 5. 

In [1]:
import sklearn
import pandas as pd
import numpy as np

print("Sklearn version:", sklearn.__version__)
print("Pandas version:", pd.__version__)
print("Numpy version:", np.__version__)

Sklearn version: 1.0.2
Pandas version: 1.4.4
Numpy version: 1.21.5


In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
from typing import Dict, List, Union, Tuple
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import cross_validate
from scipy.stats import linregress
from sklearn.impute import SimpleImputer

#pd.options.display.float_format = '{:.6f}'.format

# Utils

In [4]:
from sklearn.linear_model import PassiveAggressiveRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso

size:int = 10
ModelRegressor = Union[PassiveAggressiveRegressor, KNeighborsRegressor, DecisionTreeRegressor, Ridge, Lasso]
ModelDict = Dict[str, Tuple[ModelRegressor, Dict[str, List[float]]]]
model_dict:ModelDict = {
        "PassiveAggressiveRegressor" : (PassiveAggressiveRegressor(random_state=0), {'C': list(np.random.uniform(size=size))}),
        "KNeighborsRegressor" : (KNeighborsRegressor(), {'n_neighbors': list(range(1, 10))}),
        "DecisionTreeRegressor" : (DecisionTreeRegressor(random_state=0), {'ccp_alpha': list(np.random.uniform(size=size))}),
        "Ridge" : (Ridge(random_state=0), {'alpha': list(np.random.uniform(size=size))}),
        "Lasso" : (Lasso(random_state=0, normalize=True), {'alpha': list(np.random.uniform(size=size))})
    }

In [5]:
def generate_tabel(x:np.ndarray, y:np.ndarray, models:ModelDict) -> pd.core.frame.DataFrame:
    '''
    Generates pandas DataFrame for given atributes(:param x:) and classes(:param y:) using GridSearchCV and RandomizedSearchCV on given models
    :param x: numpy array, dataset atributes
    :param y: numpy array, dataset classes
    :param models: ModelDict, dictionary where the key is the model name and the values is a tuple formed from the model and a dictionary for hyperparameters
    :return: pandas DataFrame, generated table
    '''
    scoring_list:List[str] = ['neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_median_absolute_error']
    data_marks:pd.core.frame.DataFrame = pd.DataFrame(columns=['Model','Search_strategy'] + ['test_'+x for x in scoring_list] + ['train_'+x for x in scoring_list] + ['fit_time', 'score_time'])
    for model_name, (model, param_grid) in models.items():
        grid:GridSearchCV = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, return_train_score=True, scoring='neg_mean_squared_error')
        result:Dict[str, Union[str, float]] = cross_validate(grid, x, y, cv=5, scoring=scoring_list, return_train_score=True)
        result:Dict[str, Union[str, float]] = {key: np.mean(value) for key, value in result.items()}
        result['Model'] = model_name
        result["Search_strategy"] = "GridSearchCV"
        data_marks:pd.core.frame.DataFrame = data_marks.append(result, ignore_index=True)
        
        rand_search = RandomizedSearchCV(estimator=model, n_iter=len(param_grid), param_distributions=param_grid, return_train_score=True, scoring='neg_mean_squared_error')
        result:Dict[str, Union[str, float]] = cross_validate(rand_search, x, y, cv=5, scoring=scoring_list, return_train_score=True)
        result:Dict[str, Union[str, float]] = {key: np.mean(value) for key, value in result.items()}
        result['Model'] = model_name
        result["Search_strategy"] = "RandomizedSearchCV"
        data_marks:pd.core.frame.DataFrame = data_marks.append(result, ignore_index=True)
    return data_marks

In [6]:
def pozitive(data_frame:pd.core.frame.DataFrame) -> pd.core.frame.DataFrame:
    '''
    Sets all numerical values to positive and renames columns to show the change
    :param data_frame: pandas DataFrame, DataFrame to be modified
    :return: pandas DataFrame, modified DataFrame
    '''
    data_frame[data_frame.columns[data_frame.dtypes != object]] = data_frame[data_frame.columns[data_frame.dtypes != object]].abs()
    new_columns:Dict[str, Union[str, float]] = {x: x if '_neg_' not in x else x.replace('_neg_','_') for x in data_frame.columns}
    data_frame.rename(columns=new_columns, inplace=True)
    return data_frame

def highlight_min_max(s) -> List[str]:
    '''
    Generates a list of style strings that are going to be applied on a DataFrame based on the value being min, max or none on a given column
    :param s: iterator, iterate through DataFrame
    :return: List[str], list with given style to be apply on column
    '''
    if s.dtype == object:
        is_min:List[bool] = [False for _ in range(s.shape[0])]
        is_max:List[bool] = [False for _ in range(s.shape[0])]
    else:
        is_min:List[bool] = s == s.min()
        is_max:List[bool] = s == s.max()
    return ['background: red' if cell_max else 'background: green' if cell_min else '' for cell_max, cell_min in zip(is_max, is_min)]

In [7]:
def write_to_html_file(df:pd.core.frame.DataFrame, title:str='', filename:str='out.html') -> None:
    '''
    Write an entire dataframe to an HTML file with nice formatting.
    :param df: pandas DataFrame, DataFrame to be writen
    :param title: string, title of DataFrame
    :param filename: string, html filename
    return: None
    '''

    result:str = '''
<html>
<head>
<style>

    h2 {
        text-align: center;
        font-family: Helvetica, Arial, sans-serif;
    }
    table { 
        margin-left: auto;
        margin-right: auto;
    }
    table, th, td {
        border: 1px solid black;
        border-collapse: collapse;
    }
    th, td {
        padding: 5px;
        text-align: center;
        font-family: Helvetica, Arial, sans-serif;
        font-size: 90%;
    }
    table tbody tr:hover {
        background-color: #dddddd;
    }
    .wide {
        width: 90%; 
    }

</style>
</head>
<body>
    '''
    result += '<h2> %s </h2>\n' % title
    if type(df) == pd.io.formats.style.Styler:
        result += df.render()
    else:
        result += df.to_html(classes='wide', escape=False)
    result += '''
</body>
</html>
'''
    with open(filename, 'w') as f:
        f.write(result)

In [8]:
def run(filepath:str, header:int='infer', title:str='', filename:str='out.html') -> None:
    '''
    Main run method that prints everything
    :param filepath: str, path object or file-like object
    :param header: int, list of int, None, default ‘infer’
    :param title: string, title of DataFrame
    :param filename: string, html filename
    :return: Nothing
    '''
    data:pd.core.frame.DataFrame = pd.read_csv(filepath, header=header)
    
    values:np.ndarray = data.values
    print(f"Shape: {values.shape}")
    print(f"Missing values: {pd.isnull(values).sum()}")

    x:np.ndarray = values[:, :-1]
    y:np.ndarray = values[:, -1]
    
    df:pd.core.frame.DataFrame = generate_tabel(x, y, model_dict)
    print(df)
    
    df:pd.core.frame.DataFrame = pozitive(df)
    stylized:pd.io.formats.style.Styler = df.style.apply(highlight_min_max)
    print(stylized)
    
    write_to_html_file(stylized, title, filename)

### Forest Fires Data Set
http://archive.ics.uci.edu/ml/datasets/Forest+Fires \
Missing Values: N/A (NO)

In [9]:
forest_fires_data:pd.core.frame.DataFrame = pd.read_csv("./data/forestfires.csv", header=0)

# Dropping Month and Day columns
# Another option is to convert from string to int (Jan -> 1, Feb ->2 etc... and the same for days)
# forest_fires_data:pd.core.frame.DataFrame = forest_fires_data.drop(columns=['month', 'day'])
months_replace:Dict[str, int] = {'jan':1, 'feb':2, 'mar':3, 'apr':4, 'may':5, 'jun':6, 'jul':7, 'aug':8, 'sep':9, 'oct':10, 'nov':11, 'dec':12}
days_replace:Dict[str, int] = {'mon':1, 'tue':2, 'wed':3, 'thu':4, 'fri':5, 'sat':6, 'sun':7}
forest_fires_data.replace(months_replace, inplace=True)
forest_fires_data.replace(days_replace, inplace=True)

forest_fires_values:np.ndarray = forest_fires_data.values
print(f"Shape: {forest_fires_values.shape}")
print(f"Missing values: {pd.isnull(forest_fires_values).sum()}")

forest_fires_x:np.ndarray = forest_fires_values[:,:-1]
forest_fires_y:np.ndarray = forest_fires_values[:,-1]

Shape: (517, 13)
Missing values: 0


In [10]:
forest_fires_df:pd.core.frame.DataFrame = generate_tabel(forest_fires_x, forest_fires_y, model_dict)
forest_fires_df

,Model,Search_strategy,test_neg_mean_absolute_error,test_neg_mean_squared_error,test_neg_median_absolute_error,train_neg_mean_absolute_error,train_neg_mean_squared_error,train_neg_median_absolute_error,fit_time,score_time
0,PassiveAggressiveRegressor,GridSearchCV,-21.289600,-4385.110802,-14.696061,-22.135763,-4594.958727,-13.633892,0.083614,0.001001
1,PassiveAggressiveRegressor,RandomizedSearchCV,-21.289600,-4385.110802,-14.696061,-22.135763,-4594.958727,-13.633892,0.016398,0.001001
2,KNeighborsRegressor,GridSearchCV,-23.576418,-4858.889659,-9.662625,-16.729832,-3349.292541,-5.522194,0.135000,0.002400
3,KNeighborsRegressor,RandomizedSearchCV,-24.384729,-5206.022448,-8.497062,-15.841875,-3089.014220,-4.831412,0.029615,0.001998
4,DecisionTreeRegressor,GridSearchCV,-24.042041,-5498.660406,-4.141487,-2.652099,-16.179441,-1.890650,0.149984,0.001174
5,DecisionTreeRegressor,RandomizedSearchCV,-24.076678,-5508.468396,-4.100248,-2.339178,-13.485576,-1.668891,0.038625,0.000985
6,Ridge,GridSearchCV,-21.350781,-4180.285602,-13.662747,-19.369999,-3926.598792,-11.669593,0.063387,0.000988
7,Ridge,RandomizedSearchCV,-21.352031,-4180.366496,-13.660476,-19.371322,-3926.598387,-11.669665,0.012800,0.000800
8,Lasso,GridSearchCV,-19.486763,-4137.714942,-12.065625,-18.467043,-4005.850454,-12.199807,0.081612,0.000986
9,Lasso,RandomizedSearchCV,-19.364700,-4139.133968,-12.314993,-18.485257,-4036.321340,-12.691993,0.016043,0.001201


In [11]:
forest_fires_df:pd.core.frame.DataFrame = pozitive(forest_fires_df)
forest_fires_stylized:pd.io.formats.style.Styler = forest_fires_df.style.apply(highlight_min_max)
forest_fires_stylized

,Model,Search_strategy,test_mean_absolute_error,test_mean_squared_error,test_median_absolute_error,train_mean_absolute_error,train_mean_squared_error,train_median_absolute_error,fit_time,score_time
0,PassiveAggressiveRegressor,GridSearchCV,21.289600,4385.110802,14.696061,22.135763,4594.958727,13.633892,0.083614,0.001001
1,PassiveAggressiveRegressor,RandomizedSearchCV,21.289600,4385.110802,14.696061,22.135763,4594.958727,13.633892,0.016398,0.001001
2,KNeighborsRegressor,GridSearchCV,23.576418,4858.889659,9.662625,16.729832,3349.292541,5.522194,0.135000,0.002400
3,KNeighborsRegressor,RandomizedSearchCV,24.384729,5206.022448,8.497062,15.841875,3089.014220,4.831412,0.029615,0.001998
4,DecisionTreeRegressor,GridSearchCV,24.042041,5498.660406,4.141487,2.652099,16.179441,1.890650,0.149984,0.001174
5,DecisionTreeRegressor,RandomizedSearchCV,24.076678,5508.468396,4.100248,2.339178,13.485576,1.668891,0.038625,0.000985
6,Ridge,GridSearchCV,21.350781,4180.285602,13.662747,19.369999,3926.598792,11.669593,0.063387,0.000988
7,Ridge,RandomizedSearchCV,21.352031,4180.366496,13.660476,19.371322,3926.598387,11.669665,0.012800,0.000800
8,Lasso,GridSearchCV,19.486763,4137.714942,12.065625,18.467043,4005.850454,12.199807,0.081612,0.000986
9,Lasso,RandomizedSearchCV,19.364700,4139.133968,12.314993,18.485257,4036.321340,12.691993,0.016043,0.001201


In [12]:
write_to_html_file(forest_fires_stylized, 'Forest Fires', 'Forest_Fires.html')

### Abalone Data Set
https://www.dcc.fc.up.pt/~ltorgo/Regression/abalone.html \
https://archive.ics.uci.edu/ml/datasets/abalone \
Missing Values: NO

In [13]:
abalone_data:pd.core.frame.DataFrame = pd.read_csv("./data/abalone.data", header=None)

# Need to replace string values with numeric ones
# abalone_data.loc[abalone_data[0] == 'M', 0] = 1
# abalone_data.loc[abalone_data[0] == 'F', 0] = 2
# abalone_data.loc[abalone_data[0] == 'I', 0] = 3
abalone_data.replace({'M': 1, 'F': 2, 'I': 3}, inplace=True)

abalone_values:np.ndarray = abalone_data.values
print(f"Shape: {abalone_values.shape}")
print(f"Missing values: {pd.isnull(abalone_values).sum()}")

abalone_x:np.ndarray = abalone_values[:, :-1]
abalone_y:np.ndarray = abalone_values[:, -1]

Shape: (4177, 9)
Missing values: 0


In [14]:
abalone_df:pd.core.frame.DataFrame = generate_tabel(abalone_x, abalone_y, model_dict)
abalone_df

,Model,Search_strategy,test_neg_mean_absolute_error,test_neg_mean_squared_error,test_neg_median_absolute_error,train_neg_mean_absolute_error,train_neg_mean_squared_error,train_neg_median_absolute_error,fit_time,score_time
0,PassiveAggressiveRegressor,GridSearchCV,-1.889704,-6.727892,-1.461098,-1.749377,-5.924989,-1.309722,0.193998,0.001203
1,PassiveAggressiveRegressor,RandomizedSearchCV,-1.834496,-5.796645,-1.535310,-2.078539,-7.303202,-1.756689,0.045183,0.001014
2,KNeighborsRegressor,GridSearchCV,-1.593347,-5.258827,-1.155556,-1.363747,-3.819233,-0.977778,1.148156,0.013217
3,KNeighborsRegressor,RandomizedSearchCV,-1.741227,-6.147701,-1.266667,-1.121447,-2.592044,-0.733333,0.239600,0.011586
4,DecisionTreeRegressor,GridSearchCV,-1.917983,-6.983345,-1.526234,-1.769023,-5.929273,-1.352665,1.065022,0.001203
5,DecisionTreeRegressor,RandomizedSearchCV,-2.078996,-7.856950,-1.727651,-1.921683,-6.872720,-1.439240,0.274209,0.001003
6,Ridge,GridSearchCV,-1.648507,-5.265397,-1.237372,-1.578038,-4.791700,-1.168665,0.079788,0.001001
7,Ridge,RandomizedSearchCV,-1.648139,-5.240953,-1.238129,-1.579578,-4.801846,-1.169737,0.016388,0.001203
8,Lasso,GridSearchCV,-2.438534,-10.901912,-2.009413,-2.366977,-10.336002,-1.933683,0.099016,0.000999
9,Lasso,RandomizedSearchCV,-2.438534,-10.901912,-2.009413,-2.366977,-10.336002,-1.933683,0.021185,0.001215


In [15]:
abalone_df:pd.core.frame.DataFrame = pozitive(abalone_df)
abalone_stylized:pd.io.formats.style.Styler = abalone_df.style.apply(highlight_min_max)
abalone_stylized

,Model,Search_strategy,test_mean_absolute_error,test_mean_squared_error,test_median_absolute_error,train_mean_absolute_error,train_mean_squared_error,train_median_absolute_error,fit_time,score_time
0,PassiveAggressiveRegressor,GridSearchCV,1.889704,6.727892,1.461098,1.749377,5.924989,1.309722,0.193998,0.001203
1,PassiveAggressiveRegressor,RandomizedSearchCV,1.834496,5.796645,1.535310,2.078539,7.303202,1.756689,0.045183,0.001014
2,KNeighborsRegressor,GridSearchCV,1.593347,5.258827,1.155556,1.363747,3.819233,0.977778,1.148156,0.013217
3,KNeighborsRegressor,RandomizedSearchCV,1.741227,6.147701,1.266667,1.121447,2.592044,0.733333,0.239600,0.011586
4,DecisionTreeRegressor,GridSearchCV,1.917983,6.983345,1.526234,1.769023,5.929273,1.352665,1.065022,0.001203
5,DecisionTreeRegressor,RandomizedSearchCV,2.078996,7.856950,1.727651,1.921683,6.872720,1.439240,0.274209,0.001003
6,Ridge,GridSearchCV,1.648507,5.265397,1.237372,1.578038,4.791700,1.168665,0.079788,0.001001
7,Ridge,RandomizedSearchCV,1.648139,5.240953,1.238129,1.579578,4.801846,1.169737,0.016388,0.001203
8,Lasso,GridSearchCV,2.438534,10.901912,2.009413,2.366977,10.336002,1.933683,0.099016,0.000999
9,Lasso,RandomizedSearchCV,2.438534,10.901912,2.009413,2.366977,10.336002,1.933683,0.021185,0.001215


In [16]:
write_to_html_file(abalone_stylized, 'Abalone', 'Abalone.html')

### Auto Prices Data Set
https://www.dcc.fc.up.pt/~ltorgo/Regression/price.html \
https://archive.ics.uci.edu/ml/datasets/automobile \
Missing Values: NO (YES - on original)

In [17]:
auto_prices_data:pd.core.frame.DataFrame = pd.read_csv("./data/price.data", header=None)

auto_prices_values:np.ndarray = auto_prices_data.values
print(f"Shape: {auto_prices_values.shape}")
print(f"Missing values: {pd.isnull(auto_prices_values).sum()}")

auto_prices_x:np.ndarray = auto_prices_values[:, :-1]
auto_prices_y:np.ndarray = auto_prices_values[:, -1]

Shape: (159, 16)
Missing values: 0


In [18]:
auto_prices_df:pd.core.frame.DataFrame = generate_tabel(auto_prices_x, auto_prices_y, model_dict)
auto_prices_df

,Model,Search_strategy,test_neg_mean_absolute_error,test_neg_mean_squared_error,test_neg_median_absolute_error,train_neg_mean_absolute_error,train_neg_mean_squared_error,train_neg_median_absolute_error,fit_time,score_time
0,PassiveAggressiveRegressor,GridSearchCV,-2776.026294,-1.238992e+07,-2351.600406,-2354.102953,-9.241892e+06,-1918.672010,0.079401,0.001001
1,PassiveAggressiveRegressor,RandomizedSearchCV,-2776.026294,-1.238992e+07,-2351.600406,-2354.102953,-9.241892e+06,-1918.672010,0.013986,0.001001
2,KNeighborsRegressor,GridSearchCV,-2240.223454,-1.506107e+07,-1015.873333,-1266.600476,-3.875463e+06,-720.928889,0.137601,0.003000
3,KNeighborsRegressor,RandomizedSearchCV,-2532.784162,-1.835955e+07,-1259.355556,-992.323377,-3.169872e+06,-540.711111,0.044398,0.004419
4,DecisionTreeRegressor,GridSearchCV,-2698.300202,-2.100291e+07,-1335.700000,-34.446125,-2.536300e+04,0.000000,0.132787,0.001418
5,DecisionTreeRegressor,RandomizedSearchCV,-2698.300202,-2.100291e+07,-1335.700000,-34.446125,-2.536300e+04,0.000000,0.028413,0.001400
6,Ridge,GridSearchCV,-2449.349995,-1.305915e+07,-1740.763012,-1561.211522,-4.504249e+06,-1110.056642,0.060602,0.000998
7,Ridge,RandomizedSearchCV,-2465.384650,-1.319175e+07,-1740.899192,-1555.586872,-4.495220e+06,-1111.109721,0.011999,0.000800
8,Lasso,GridSearchCV,-2441.537037,-1.273652e+07,-1762.304456,-1548.917094,-4.509872e+06,-1083.444327,0.135386,0.001003
9,Lasso,RandomizedSearchCV,-2449.779640,-1.290040e+07,-1757.299320,-1550.352012,-4.502178e+06,-1073.522013,0.021598,0.001001


In [19]:
auto_prices_df:pd.core.frame.DataFrame = pozitive(auto_prices_df)
auto_prices_stylized:pd.io.formats.style.Styler = auto_prices_df.style.apply(highlight_min_max)
auto_prices_stylized

,Model,Search_strategy,test_mean_absolute_error,test_mean_squared_error,test_median_absolute_error,train_mean_absolute_error,train_mean_squared_error,train_median_absolute_error,fit_time,score_time
0,PassiveAggressiveRegressor,GridSearchCV,2776.026294,12389915.388058,2351.600406,2354.102953,9241892.318235,1918.672010,0.079401,0.001001
1,PassiveAggressiveRegressor,RandomizedSearchCV,2776.026294,12389915.388058,2351.600406,2354.102953,9241892.318235,1918.672010,0.013986,0.001001
2,KNeighborsRegressor,GridSearchCV,2240.223454,15061073.646750,1015.873333,1266.600476,3875462.998006,720.928889,0.137601,0.003000
3,KNeighborsRegressor,RandomizedSearchCV,2532.784162,18359545.012211,1259.355556,992.323377,3169871.733943,540.711111,0.044398,0.004419
4,DecisionTreeRegressor,GridSearchCV,2698.300202,21002914.328831,1335.700000,34.446125,25363.004497,0.000000,0.132787,0.001418
5,DecisionTreeRegressor,RandomizedSearchCV,2698.300202,21002914.328831,1335.700000,34.446125,25363.004497,0.000000,0.028413,0.001400
6,Ridge,GridSearchCV,2449.349995,13059149.054423,1740.763012,1561.211522,4504249.439012,1110.056642,0.060602,0.000998
7,Ridge,RandomizedSearchCV,2465.384650,13191750.934521,1740.899192,1555.586872,4495219.971349,1111.109721,0.011999,0.000800
8,Lasso,GridSearchCV,2441.537037,12736524.993229,1762.304456,1548.917094,4509872.103315,1083.444327,0.135386,0.001003
9,Lasso,RandomizedSearchCV,2449.779640,12900401.596550,1757.299320,1550.352012,4502177.785472,1073.522013,0.021598,0.001001


In [20]:
write_to_html_file(auto_prices_stylized, 'Auto Prices', 'Auto_Prices.html')

### Pole Telecomm Data Set
https://www.dcc.fc.up.pt/~ltorgo/Regression/pole.html \
Missing Values: NO

In [21]:
pole_data:pd.core.frame.DataFrame = pd.read_csv("./data/pol.data", header=None)

pole_values:np.ndarray = pole_data.values
print(f"Shape: {pole_values.shape}")
print(f"Missing values: {pd.isnull(pole_values).sum()}")

pole_x:np.ndarray = pole_values[:, :-1]
pole_y:np.ndarray = pole_values[:, -1]

Shape: (5000, 49)
Missing values: 0


In [22]:
pole_df:pd.core.frame.DataFrame = generate_tabel(pole_x, pole_y, model_dict)
pole_df

,Model,Search_strategy,test_neg_mean_absolute_error,test_neg_mean_squared_error,test_neg_median_absolute_error,train_neg_mean_absolute_error,train_neg_mean_squared_error,train_neg_median_absolute_error,fit_time,score_time
0,PassiveAggressiveRegressor,GridSearchCV,-35.418334,-1895.796234,-33.418444,-34.964737,-1849.177855,-32.942874,0.489211,0.001600
1,PassiveAggressiveRegressor,RandomizedSearchCV,-35.418334,-1895.796234,-33.418444,-34.964737,-1849.177855,-32.942874,0.179601,0.001601
2,KNeighborsRegressor,GridSearchCV,-3.751000,-99.727222,0.000000,-2.192083,-38.283194,0.000000,14.622134,0.159583
3,KNeighborsRegressor,RandomizedSearchCV,-4.027644,-110.759798,0.000000,-2.281244,-44.780633,0.000000,3.121302,0.156387
4,DecisionTreeRegressor,GridSearchCV,-3.452258,-87.075659,-0.436580,-2.109361,-25.147511,-0.437414,0.986202,0.001412
5,DecisionTreeRegressor,RandomizedSearchCV,-3.837240,-92.931088,-0.957114,-2.800447,-39.479444,-0.950647,0.198799,0.001399
6,Ridge,GridSearchCV,-26.737172,-942.780129,-27.974892,-26.563892,-929.992553,-27.899849,0.275989,0.001200
7,Ridge,RandomizedSearchCV,-26.737159,-942.783533,-27.971598,-26.563847,-929.992352,-27.901510,0.059004,0.001598
8,Lasso,GridSearchCV,-35.577894,-1595.914441,-32.634125,-35.571892,-1595.299450,-32.697751,0.349602,0.001006
9,Lasso,RandomizedSearchCV,-37.511347,-1753.468618,-29.308000,-37.503131,-1752.638083,-29.308000,0.083770,0.001603


In [23]:
pole_df:pd.core.frame.DataFrame = pozitive(pole_df)
pole_stylized:pd.io.formats.style.Styler = pole_df.style.apply(highlight_min_max)
pole_stylized

,Model,Search_strategy,test_mean_absolute_error,test_mean_squared_error,test_median_absolute_error,train_mean_absolute_error,train_mean_squared_error,train_median_absolute_error,fit_time,score_time
0,PassiveAggressiveRegressor,GridSearchCV,35.418334,1895.796234,33.418444,34.964737,1849.177855,32.942874,0.489211,0.001600
1,PassiveAggressiveRegressor,RandomizedSearchCV,35.418334,1895.796234,33.418444,34.964737,1849.177855,32.942874,0.179601,0.001601
2,KNeighborsRegressor,GridSearchCV,3.751000,99.727222,0.000000,2.192083,38.283194,0.000000,14.622134,0.159583
3,KNeighborsRegressor,RandomizedSearchCV,4.027644,110.759798,0.000000,2.281244,44.780633,0.000000,3.121302,0.156387
4,DecisionTreeRegressor,GridSearchCV,3.452258,87.075659,0.436580,2.109361,25.147511,0.437414,0.986202,0.001412
5,DecisionTreeRegressor,RandomizedSearchCV,3.837240,92.931088,0.957114,2.800447,39.479444,0.950647,0.198799,0.001399
6,Ridge,GridSearchCV,26.737172,942.780129,27.974892,26.563892,929.992553,27.899849,0.275989,0.001200
7,Ridge,RandomizedSearchCV,26.737159,942.783533,27.971598,26.563847,929.992352,27.901510,0.059004,0.001598
8,Lasso,GridSearchCV,35.577894,1595.914441,32.634125,35.571892,1595.299450,32.697751,0.349602,0.001006
9,Lasso,RandomizedSearchCV,37.511347,1753.468618,29.308000,37.503131,1752.638083,29.308000,0.083770,0.001603


In [24]:
write_to_html_file(pole_stylized, 'Pole Telecomm', 'Pole_Telecomm.html')

### Computer Activity Data Set
https://www.dcc.fc.up.pt/~ltorgo/Regression/comp.html \
Missing Values: NO

In [25]:
computer_activity_data:pd.core.frame.DataFrame = pd.read_csv("./data/cpu_small.data", header=None)

computer_activity_values:np.ndarray = computer_activity_data.values
print(f"Shape: {computer_activity_values.shape}")
print(f"Missing values: {pd.isnull(computer_activity_values).sum()}")

computer_activity_x:np.ndarray = computer_activity_values[:, :-1]
computer_activity_y:np.ndarray = computer_activity_values[:, -1]

Shape: (8192, 13)
Missing values: 0


In [26]:
computer_activity_df:pd.core.frame.DataFrame = generate_tabel(computer_activity_x, computer_activity_y, model_dict)
computer_activity_df

,Model,Search_strategy,test_neg_mean_absolute_error,test_neg_mean_squared_error,test_neg_median_absolute_error,train_neg_mean_absolute_error,train_neg_mean_squared_error,train_neg_median_absolute_error,fit_time,score_time
0,PassiveAggressiveRegressor,GridSearchCV,-18.509471,-520.031831,-15.804549,-18.434900,-520.855234,-15.549557,0.493599,0.001200
1,PassiveAggressiveRegressor,RandomizedSearchCV,-18.509471,-520.031831,-15.804549,-18.434900,-520.855234,-15.549557,0.091199,0.001415
2,KNeighborsRegressor,GridSearchCV,-4.553253,-42.378975,-3.133333,-4.063922,-33.803513,-2.777778,2.576850,0.024976
3,KNeighborsRegressor,RandomizedSearchCV,-4.686228,-45.902205,-3.194444,-3.710089,-29.100670,-2.505556,0.685210,0.023988
4,DecisionTreeRegressor,GridSearchCV,-2.803906,-15.938500,-2.060940,-2.692436,-13.690082,-2.035630,4.401769,0.001202
5,DecisionTreeRegressor,RandomizedSearchCV,-3.174808,-19.793935,-2.191333,-3.120613,-18.123219,-2.232803,1.035396,0.001403
6,Ridge,GridSearchCV,-6.178168,-97.440392,-4.436230,-6.158985,-96.246212,-4.412110,0.111414,0.001196
7,Ridge,RandomizedSearchCV,-6.178169,-97.440400,-4.436193,-6.158984,-96.246212,-4.412080,0.024575,0.001211
8,Lasso,GridSearchCV,-10.631039,-338.721199,-7.600052,-10.630539,-338.572544,-7.916683,0.149582,0.001201
9,Lasso,RandomizedSearchCV,-10.631039,-338.721199,-7.600052,-10.630539,-338.572544,-7.916683,0.032802,0.001001


In [27]:
computer_activity_df:pd.core.frame.DataFrame = pozitive(computer_activity_df)
computer_activity_stylized:pd.io.formats.style.Styler = computer_activity_df.style.apply(highlight_min_max)
computer_activity_stylized

,Model,Search_strategy,test_mean_absolute_error,test_mean_squared_error,test_median_absolute_error,train_mean_absolute_error,train_mean_squared_error,train_median_absolute_error,fit_time,score_time
0,PassiveAggressiveRegressor,GridSearchCV,18.509471,520.031831,15.804549,18.434900,520.855234,15.549557,0.493599,0.001200
1,PassiveAggressiveRegressor,RandomizedSearchCV,18.509471,520.031831,15.804549,18.434900,520.855234,15.549557,0.091199,0.001415
2,KNeighborsRegressor,GridSearchCV,4.553253,42.378975,3.133333,4.063922,33.803513,2.777778,2.576850,0.024976
3,KNeighborsRegressor,RandomizedSearchCV,4.686228,45.902205,3.194444,3.710089,29.100670,2.505556,0.685210,0.023988
4,DecisionTreeRegressor,GridSearchCV,2.803906,15.938500,2.060940,2.692436,13.690082,2.035630,4.401769,0.001202
5,DecisionTreeRegressor,RandomizedSearchCV,3.174808,19.793935,2.191333,3.120613,18.123219,2.232803,1.035396,0.001403
6,Ridge,GridSearchCV,6.178168,97.440392,4.436230,6.158985,96.246212,4.412110,0.111414,0.001196
7,Ridge,RandomizedSearchCV,6.178169,97.440400,4.436193,6.158984,96.246212,4.412080,0.024575,0.001211
8,Lasso,GridSearchCV,10.631039,338.721199,7.600052,10.630539,338.572544,7.916683,0.149582,0.001201
9,Lasso,RandomizedSearchCV,10.631039,338.721199,7.600052,10.630539,338.572544,7.916683,0.032802,0.001001


In [28]:
write_to_html_file(computer_activity_stylized, 'Computer Activity', 'Computer_Activity.html')

### Block 1
```python
_data:pd.core.frame.DataFrame = pd.read_csv("./data/", header=None)

_values:np.ndarray = _data.values
print(f"Shape: {_values.shape}")
print(f"Missing values: {pd.isnull(_values).sum()}")

_x:np.ndarray = _values[:, :-1]
_y:np.ndarray = _values[:, -1]
```

---

### Block 2
```python
_df:pd.core.frame.DataFrame = generate_tabel(_x, _y, model_dict)
_df
```

---

### Block 3
```python
_df:pd.core.frame.DataFrame = pozitive(_df)
_stylized:pd.io.formats.style.Styler = _df.style.apply(highlight_min_max)
_stylized
```

---

### Block 4
```python
write_to_html_file(_stylized, '', '.html')
```

---
---
---

# Documentation

## PassiveAggressiveRegressor

Algoritmul de regresie se bazează pe o funcție de pierdere Hinge ușor diferită (numită $\epsilon$-insensitive):

<img src="images/PassiveAggressive1.png">

Parametrul ε determină o toleranță pentru erorile de predicție. Condițiile de actualizare sunt aceleași adoptate pentru problemele de clasificare, iar regula de actualizare rezultată este:
<img src="images/PassiveAggressive2.png" width="450">

Algoritmii Passive Aggressive sunt o familie de algoritmi pentru învățarea pe scară largă. Aceștia sunt asemănători cu Perceptronul prin faptul că nu necesită o rată de învățare. Cu toate acestea, spre deosebire de Perceptron, ei includ un parametru de regularizare C sau dimensiunea maximă a pasului. Valoarea implicită este 1,0.\
Pentru regresie, PassiveAgressiveRegressor poate fi utilizat cu loss='epsilon_insensitive' (PA-I) sau loss = 'squared_epsilon_insensitive' (PA-II).

Calitatea regresiei (în special, durata perioadei tranzitorii în care eroarea este mare) poate fi controlată prin alegerea unor valori mai bune pentru C și ε ( epsilon ). În special, se recomandă să se verifice diferite centre de interval pentru C (100, 10, 1, 0,1, 0,01), pentru a determina dacă este preferabilă o agresivitate mai mare.\
Parametrul ε ( epsilon ) determină diferența dintre predicția curentă și eticheta corectă. Dacă este sub un anumit prag, modelul nu este actualizat. Valoarea implicită este 0,1.

De notat în legatură cu parametrul epsilon, este ca dorim să actualizăm modelul doar dacă algoritmul ajunge la o greseală foarte mare.\
Valorile mai mari ale lui C produc o agresivitate mai puternică (cu un risc mai mare de destabilizare în prezența zgomotului), în timp ce valorile mai mici permit o mai bună adaptare.

Algoritmii Passive Aggressive sunt algoritmi de machine learning aplicați într-un mediu online sau leneș. O setare online sau leneșă înseamnă că algoritmul își procesează datele de învățare sample cu sample într-o secvență serială.

<img src="images/PassiveAggressive3.png" width="600">

### Sources
https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.PassiveAggressiveRegressor.html  
https://www.bonaccorso.eu/2017/10/06/ml-algorithms-addendum-passive-aggressive-algorithms/  
https://koaning.io/posts/passive-agressive-algorithms/  

## KNeighborsRegressor

În regresia KNN, ținta este prezisă prin interpolarea locală a țintelor asociate celor mai apropiați vecini din setul de formare. Cu alte cuvinte, regresia KNN calculează media obiectivelor numerice ale celor mai apropiați K vecini. O altă abordare utilizează o medie ponderată în funcție de distanța inversă a celor mai apropiați K vecini. Regresia KNN utilizează aceleași funcții de distanță ca și clasificarea KNN.

Atunci când se configurează un model KNN, există doar câțiva parametri care trebuie aleși/pot fi modificați pentru a îmbunătăți performanța:
- numărul de vecini: După cum s-a discutat, creșterea numărului de n_vecini va tinde să netezească limitele deciziei, evitând overfitting cu prețul unei anumite rezoluții.
- metric: După cum se pare, există diferite moduri de a măsura cât de "apropiate" sunt două puncte unul de celălalt, iar diferențele dintre aceste metode pot deveni semnificative în dimensiuni mai mari. Cea mai frecvent utilizată este distanța euclidiană, tipul standard pe care probabil că l-ați învățat în școala generală folosind teorema lui Pitagora. O altă metrică este așa-numita "distanță Manhattan", care măsoară distanța parcursă în fiecare direcție cardinală, mai degrabă decât de-a lungul diagonalei (ca și cum ați merge de la o intersecție de străzi din Manhattan la alta și ar trebui să urmați grila străzilor, în loc să puteți lua cel mai scurt traseu "în linie dreaptă"). Mai general, acestea sunt de fapt ambele forme ale ceea ce se numește "distanța Minkowski":
<img src="images/KNN1.png" width="300">
- weights: O modalitate de a rezolva atât problema unei posibile "egalități" atunci când algoritmul decide asupra unei clase, cât și problema în care predicțiile noastre de regresie s-au înrăutățit spre marginile setului de date este introducerea ponderării. Cu ajutorul ponderilor, punctele apropiate vor conta mai mult decât punctele mai îndepărtate. Algoritmul se va uita în continuare la toți cei k vecini apropiați, dar vecinii mai apropiați vor avea un vot mai important decât cei mai îndepărtați. Aceasta nu este o soluție perfectă și aduce din nou în discuție posibilitatea de overfitting.

Pentru seturi mari de date, KNN poate fi, prin urmare, o metodă relativ lentă în comparație cu alte modele de regresie care pot necesita mai mult timp pentru a se potrivi, dar care apoi își fac predicțiile cu calcule relativ simple.\
De asemenea, KNeighbors Regression este un algoritm de machine learning aplicat într-un cadru online sau leneș.\
Regresia KNearest Neighbors (kNN) este definită prin:
<img src="images/KNN2.png" width="200">

### Example:
<img src="images/KNN3.png" width="600">

### Sources
https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsRegressor.html  
https://jmlr.org/papers/volume14/guyader13a/guyader13a.pdf  
https://www.analyticsvidhya.com/blog/2018/08/k-nearest-neighbor-introduction-regression-python/  
https://www.saedsayad.com/k_nearest_neighbors_reg.htm  

## DecisionTreeRegressor

Regresia arborelui de decizie este un model care se foloseste de o tehnică de învățare automată neliniara și non-continuu\
Divizarea în arborele de regresie se face în funcție de pierderea în eroarea medie pătratică. Aici $\overline{y}$ pentru o anumită regiune este media tuturor 	$y_{i}$ din regiunea respectivă\
Pierderea în arborii de regresie este reprezentată astfel:

<img src="images/DecisionTree1.png" width="200">

Arborele de decizie este utilizat pentru a ajusta o curbă sinusoidală cu observația zgomotoasă suplimentară. Ca urmare, se învață regresii liniare locale care aproximează curba sinusoidală.\
Putem observa că, dacă adâncimea maximă a arborelui (controlată de parametrul max_depth) este setată prea mare, arborii de decizie învață detalii prea fine ale datelor de instruire și învață din zgomot, adică face overfitting.

Decizia de a face diviziuni strategice afectează în mare măsură acuratețea unui arbore. Regresia arborilor de decizie utilizează în mod normal mean squared error (MSE) pentru a decide împărțirea unui nod în două sau mai multe subnoduri.\
Arborele de decizie și metodele bazate pe arborele general sunt un tip de algoritm de învățare supravegheată (având o variabilă țintă predefinită) și sunt utilizate în probleme de regresie dacă și numai dacă variabila țintă se află în intervalul de valori pe care le-au văzut în setul de date de instruire.

Regresia arborelui de decizie observă caracteristicile unui obiect și antrenează un model în structura unui arbore pentru a prezice date în viitor și a produce rezultate continue semnificative. Ieșirea continuă înseamnă că ieșirea/rezultatul nu este discretă, adică nu este reprezentată doar de un set discret și cunoscut de numere sau valori.

Pentru arborii de regresie, două măsuri comune de impuritate sunt:
- Least squares: această metodă este similară cu minimizarea celor mai mici pătrate într-un model liniar. Diviziunile sunt alese pentru a minimiza suma reziduală a pătratelor dintre observație și medie în fiecare nod.
- Least absolute deviations: această metodă minimizează abaterea medie absolută de la mediana dintr-un nod. Avantajul acestei metode față de cea a celor mai mici pătrate este că nu este la fel de sensibilă la valorile aberante și oferă un model mai robust. Dezavantajul constă în insensibilitatea în cazul seturilor de date care conțin o proporție mare de zerouri.

<img src="images/DecisionTree2.png" width="400">

### Sources
https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html  
https://arifromadhan19.medium.com/regrssion-in-decision-tree-a-step-by-step-cart-classification-and-regression-tree-196c6ac9711e  
https://sefiks.com/2018/08/28/a-step-by-step-regression-decision-tree-example/  
https://gdcoder.com/decision-tree-regressor-explained-in-depth/  

## Ridge

Ridge regression addresses some of the problems of Ordinary Least Squares by imposing a penalty on the size of the coefficients. The ridge coefficients minimize a penalized residual sum of squares:

$\displaystyle \min_{w} ||Xw - y||_{2}^{2} + \alpha||w||_{2}^{2}$

The complexity parameter $\alpha \ge 0$ controls the amount of shrinkage: the larger the value of , the greater the amount of shrinkage and thus the coefficients become more robust to collinearity.

<img src="images/Ridge1.png" width="500">

Regresia Ridge este o tehnică de analiză a datelor de regresie multiplă care suferă de multicoliniaritate. Atunci când apare multicoliniaritatea, estimările prin metoda celor mai mici pătrate sunt lipsite de distorsiuni, dar varianțele lor sunt mari, astfel încât pot fi departe de valoarea reală. Adăugând un grad de distorsiune estimărilor de regresie, regresia cu creastă reduce erorile standard. Se speră că efectul net va fi acela de a oferi estimări mai fiabile.

Regresia Ridge efectuează o regularizare $L2$ care nu duce la eliminarea coeficienților sau la modele rarefiate.

În cadrul regresiei ridge, primul pas este standardizarea variabilelor (atât a celor dependente, cât și a celor independente) prin scăderea mediilor acestora și împărțirea la abaterile lor standard. Acest lucru cauzează o dificultate în notație, deoarece trebuie să indicăm cumva dacă variabilele dintr-o anumită formulă sunt standardizate sau nu.

În ceea ce privește standardizarea, toate calculele de regresie ridge se bazează pe variabile standardizate. Atunci când sunt afișați coeficienții de regresie finali, aceștia sunt ajustați înapoi în scala lor inițială. Cu toate acestea, "the ridge trace" este la o scară standardizată.

În cazul regresiei ridge, puteți regla parametrul lambda astfel încât coeficienții modelului să se schimbe și, de asemenea, regresia ridge acordă ponderi diferite de importanță caracteristicilor, dar nu renunță la caracteristicile neimportante.

<img src="images/Ridge2.png" width="300">

Aplicând formula matricei pe care am văzut-o anterior, $\lambda$ ajunge la numitor. Aceasta înseamnă că, dacă mărim valoarea $\lambda$, ar trebui să scadă $\beta^{ridge}$. Dar $\beta^{ridge}$ nu pot fi zero, indiferent cât de mare este stabilită valoarea $\lambda$.

Un parametru de ajustare ($\lambda$) controlează intensitatea termenului de regularizare. Atunci când $\lambda = 0$, Ridge regression este egală cu regresia cu cele mai mici pătrate. Dacă $\lambda = \infty$, toți coeficienții sunt reduși la zero. Prin urmare, penalizarea ideală se situează undeva între $0$ și $\infty$.
    
<img src="images/Ridge3.png" width="400">

### Sources
https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html  
https://towardsdatascience.com/ridge-regression-for-better-usage-2f19b3a202db  
https://ncss-wpengine.netdna-ssl.com/wp-content/themes/ncss/pdf/Procedures/NCSS/Ridge_Regression.pdf  
https://scikit-learn.org/stable/modules/linear_model.html  
 https://www.statisticshowto.com/ridge-regression/  

## Lasso

Acronimul "LASSO" este abrevierea de la Least Absolute Shrinkage and Selection Operator.

Regresia Lasso este un tip de regresie liniară care utilizează micșorarea. Prin micșorare, valorile datelor sunt reduse spre un punct central, cum ar fi media. Procedura lasso încurajează modelele simple și rare (adică modele cu mai puțini parametri). Acest tip special de regresie este potrivit pentru modelele care prezintă niveluri ridicate de multicoliniaritate sau atunci când doriți să automatizați anumite părți ale selecției modelului, cum ar fi selecția variabilelor/eliminarea parametrilor.

Regresia Lasso efectuează regularizarea $L1$, care adaugă o penalizare egală cu valoarea absolută a magnitudinii coeficienților. Acest tip de regularizare poate avea ca rezultat modele rarefiate cu puțini coeficienți; unii coeficienți pot deveni zero și pot fi eliminați din model. Penalizările mai mari au ca rezultat valori ale coeficienților mai apropiate de zero, ceea ce este ideal pentru a produce modele mai simple.

Funcția de cost pentru Lasso. Spre deosebire de regresia ridge, nu există o soluție analitică pentru lasso, deoarece soluția este neliniară în $y$

<img src="images/Lasso1.png" width="450">

Un parametru de reglare, $\lambda$, controlează intensitatea penalizării $L1$. $\lambda$ reprezintă, în principiu, valoarea micșorării:  
Atunci când $\lambda = 0$, nu se elimină niciun parametru. Estimarea este egală cu cea obținută prin regresie liniară.  
Pe măsură ce $\lambda$ crește, din ce în ce mai mulți coeficienți sunt stabiliți la zero și eliminați (teoretic, când $\lambda = \infty$, toți coeficienții sunt eliminați).  
Pe măsură ce $\lambda$ crește, crește și distorsiunea.  
Pe măsură ce $\lambda$ scade, varianța crește.  

Un alt parametru de ajustare fină, alpha este o constantă care multiplică termenul $L1$. Valoarea implicită este de $1,0$.  
De reținut este că $alpha = 0$ este echivalent cu un minim pătrat obișnuit, rezolvat de obiectul LinearRegression. Din motive numerice, nu se recomandă utilizarea lui $alpha = 0$ cu obiectul Lasso. De asemenea, chiar și la un $alpha$ mai mic, coeficienții se reduc la zero absolut. Prin urmare, regresia lasso este o alegere mai bună atunci când avem un număr mai mare de caracteristici, deoarece face automat selecția caracteristicilor, reducând în același timp coeficienții celorlalte caracteristici la zero.

Principala problemă cu regresia lasso este că, atunci când avem variabile corelate, aceasta păstrează doar o singură variabilă și stabilește la zero alte variabile corelate. Acest lucru va duce, eventual, la o anumită pierdere de informații, ceea ce va duce la o acuratețe mai mică a modelului nostru.

Pentru inferența care utilizează estimatorul lasso, au fost propuși diverși estimatori de erori standard:
- Tibshirani (1996) a sugerat bootstrap (Efron, 1979) pentru estimarea erorilor standard și a derivat o estimare aproximativă în formă închisă.
- Fan și Li (2001) au derivat formula sandwich în cadrul probabilității ca un estimator pentru covarianța estimărilor.

"Bayesian lasso" al lui Park și Casella (2008) oferă erori standard valide pentru și oferă estimări punctuale mai stabile prin utilizarea medianei posterioare. Estimarea lasso este echivalentă cu modul distribuției posterioare în condițiile unei probabilități normale și a unei priorități independente Laplace (dublu exponențiale):

<img src="images/Lasso2.png" width="200">

<img src="images/Lasso3.png" width="550">

<img src="images/Lasso4.png" width="500">

### Sources
https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html  
https://www.analyticsvidhya.com/blog/2017/06/a-comprehensive-guide-for-linear-ridge-and-lasso-regression/  
https://www.mygreatlearning.com/blog/understanding-of-lasso-regression/  
https://online.stat.psu.edu/stat508/lesson/5/5.4  
https://machinelearningmastery.com/lasso-regression-with-python/  